In [1]:
import pandas as pd
import numpy as np
import statsmodels.regression.linear_model as sm
from sklearn.preprocessing import PolynomialFeatures
from statsmodels.tools.tools import add_constant

### Linear Regression

In [58]:
returns = pd.read_csv("/Users/luzhang/Documents/GitHub/WrappingUp_Exploring_Intraday_Momentum_2rd_try/Data/SPY_Returns_with_regime.csv")

In [59]:
returns = add_constant(returns)
returns

,const,datetime,R_LH,R_LH0,R_ROD,R_ON,R_FH,R_ONFH,R_M,R_SLH,regime
0,1.0,2007-06-15,-0.000783,-0.000785,0.003012,0.003471,-0.000065,0.003405,0.000131,-0.000522,0
1,1.0,2007-06-18,-0.000458,-0.000783,-0.000523,0.002025,-0.001043,0.000980,-0.000914,-0.000588,0
2,1.0,2007-07-10,-0.003500,-0.000914,-0.010843,-0.005422,-0.001511,-0.006924,-0.003420,-0.000528,0
3,1.0,2007-08-01,0.014601,-0.009672,-0.006121,-0.001307,0.000482,-0.000825,-0.005438,0.000138,1
4,1.0,2007-08-02,-0.002303,0.014601,0.006889,0.001023,0.003339,0.004365,-0.006180,0.008746,0
...,...,...,...,...,...,...,...,...,...,...,...
2768,1.0,2018-10-25,-0.004533,-0.007299,0.023188,0.008106,0.002618,0.010746,0.011975,0.000331,0
2769,1.0,2018-10-26,0.000189,-0.004533,-0.018139,-0.015622,0.000075,-0.015547,-0.005528,0.002911,1
2770,1.0,2018-10-29,0.004991,0.000189,-0.010554,0.013155,0.000781,0.013947,-0.016543,-0.007749,0
2771,1.0,2018-10-30,0.002639,0.004991,0.012528,-0.000493,0.012250,0.011751,-0.002286,0.003061,0


### regime 0

In [60]:
reg0 = returns[returns.regime==0]

In [61]:
reg0

,const,datetime,R_LH,R_LH0,R_ROD,R_ON,R_FH,R_ONFH,R_M,R_SLH,regime
0,1.0,2007-06-15,-0.000783,-0.000785,0.003012,0.003471,-0.000065,0.003405,0.000131,-0.000522,0
1,1.0,2007-06-18,-0.000458,-0.000783,-0.000523,0.002025,-0.001043,0.000980,-0.000914,-0.000588,0
2,1.0,2007-07-10,-0.003500,-0.000914,-0.010843,-0.005422,-0.001511,-0.006924,-0.003420,-0.000528,0
4,1.0,2007-08-02,-0.002303,0.014601,0.006889,0.001023,0.003339,0.004365,-0.006180,0.008746,0
5,1.0,2007-09-24,-0.001515,-0.001705,-0.003218,0.001051,-0.003018,-0.001970,-0.001908,0.000659,0
...,...,...,...,...,...,...,...,...,...,...,...
2767,1.0,2018-10-24,-0.007299,-0.002771,-0.023287,-0.000768,-0.001573,-0.002340,-0.013558,-0.007541,0
2768,1.0,2018-10-25,-0.004533,-0.007299,0.023188,0.008106,0.002618,0.010746,0.011975,0.000331,0
2770,1.0,2018-10-29,0.004991,0.000189,-0.010554,0.013155,0.000781,0.013947,-0.016543,-0.007749,0
2771,1.0,2018-10-30,0.002639,0.004991,0.012528,-0.000493,0.012250,0.011751,-0.002286,0.003061,0


In [62]:
y = reg0.R_LH
X = reg0[["R_LH0","R_ROD","R_ON","R_FH","R_ONFH","R_M", "R_SLH"]]

In [63]:
# define a function find the features after backward Elimination
def BackwardEli(X,y):
    cols = list(X.columns)
    pmax = 1
    while (len(cols)>0):
        p = []
        X_1 = X[cols]
        X_1 = add_constant(X_1)
        model = sm.OLS(y,X_1).fit()
        p = pd.Series(model.pvalues.values[1:],index=cols)
        pmax = max(p)
        feature_with_p_max = p.idxmax()
        if(pmax>0.05):
            cols.remove(feature_with_p_max)
        else:
            break
    return cols

In [64]:
cols = BackwardEli(X,y)
final_input_X = add_constant(reg0[cols])
final0 = sm.OLS(y,final_input_X).fit()
final0.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                   R_LH   R-squared:                       0.018
Model:                            OLS   Adj. R-squared:                  0.017
Method:                 Least Squares   F-statistic:                     12.06
Date:                Thu, 12 Oct 2023   Prob (F-statistic):           1.03e-09
Time:                        17:54:25   Log-Likelihood:                 11916.
No. Observations:                2579   AIC:                        -2.382e+04
Df Residuals:                    2574   BIC:                        -2.379e+04
Df Model:                           4                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const      -5.263e-05   4.71e-05     -1.117      0.264      -0.000    3.98e-05
R_LH0         -0.0438      0.019     -2.260      0.024      -0.082      -0.006
R_ON           0.0444      0.009      5.128      0.000       0.027       0.061
R_M            0.0261      0.008      3.174      0.002       0.010       0.042
R_SLH          0.0510      0.025      2.076      0.038       0.003       0.099
==============================================================================
Omnibus:                      326.168   Durbin-Watson:                   1.968
Prob(Omnibus):                  0.000   Jarque-Bera (JB):             3022.416
Skew:                          -0.236   Prob(JB):                         0.00
Kurtosis:                       8.282   Cond. No.                         523.
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

### regime 1

In [65]:
reg1 = returns[returns.regime==1]
reg1

,const,datetime,R_LH,R_LH0,R_ROD,R_ON,R_FH,R_ONFH,R_M,R_SLH,regime
3,1.0,2007-08-01,0.014601,-0.009672,-0.006121,-0.001307,0.000482,-0.000825,-0.005438,0.000138,1
38,1.0,2007-11-08,0.000271,-0.006931,-0.000610,0.002710,-0.001892,0.000813,-0.008734,0.007377,1
39,1.0,2007-11-09,-0.013514,0.000271,-0.001830,-0.013692,0.000962,-0.012743,0.008720,0.002314,1
78,1.0,2008-01-10,-0.005331,0.007587,0.012644,-0.007672,0.002291,-0.005398,0.015069,0.003025,1
83,1.0,2008-01-17,-0.003068,-0.010390,-0.025592,0.003135,0.000000,0.003135,-0.021733,-0.007058,1
...,...,...,...,...,...,...,...,...,...,...,...
2590,1.0,2018-02-09,0.009568,-0.015848,0.005704,0.011951,0.000192,0.012145,-0.009124,0.002786,1
2603,1.0,2018-03-01,-0.002348,-0.006990,-0.010946,0.000295,-0.004090,-0.003796,-0.011617,0.004492,1
2622,1.0,2018-03-28,-0.002342,-0.000767,-0.000307,0.000844,0.001726,0.002572,-0.003292,0.000423,1
2758,1.0,2018-10-11,-0.008702,-0.010063,-0.012690,-0.003918,0.006135,0.002193,-0.022563,0.007890,1


In [66]:
y = reg1.R_LH
X = reg1[["R_LH0","R_ROD","R_ON","R_FH","R_ONFH","R_M", "R_SLH"]]

In [67]:
cols = BackwardEli(X,y)
final_input_X = add_constant(reg1[cols])
final1 = sm.OLS(y,final_input_X).fit()
final1.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                   R_LH   R-squared:                       0.152
Model:                            OLS   Adj. R-squared:                  0.134
Method:                 Least Squares   F-statistic:                     8.465
Date:                Thu, 12 Oct 2023   Prob (F-statistic):           2.65e-06
Time:                        17:56:52   Log-Likelihood:                 606.76
No. Observations:                 194   AIC:                            -1204.
Df Residuals:                     189   BIC:                            -1187.
Df Model:                           4                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const      -3.821e-05      0.001     -0.048      0.962      -0.002       0.002
R_LH0         -0.2026      0.069     -2.939      0.004      -0.339      -0.067
R_FH          -0.1886      0.086     -2.181      0.030      -0.359      -0.018
R_ONFH         0.1251      0.049      2.553      0.011       0.028       0.222
R_SLH          0.2353      0.081      2.907      0.004       0.076       0.395
==============================================================================
Omnibus:                       12.932   Durbin-Watson:                   1.999
Prob(Omnibus):                  0.002   Jarque-Bera (JB):               16.726
Skew:                          -0.460   Prob(JB):                     0.000233
Kurtosis:                       4.105   Cond. No.                         113.
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""